Задача

1. Проанализировать тест между группами 0 и 3 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?
2. Проанализировать тест между группами 1 и 2 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?

In [11]:
# Запрос библиотек.

import pandahouse
import pandas as pd
import seaborn as sns
import numpy as np

from scipy import stats

In [12]:
# Создание соединения

connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator'
}


In [4]:
# Запрос данных (пользователи, их просмотры и лайки из групп 0 и 3
#  во временной интервал с 01.01.23 по 07.01.23 включительно)

q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM simulator_20230120.feed_actions 
WHERE toDate(time) between '2023-01-01' and '2023-01-07'
    and exp_group in (0,3)
GROUP BY exp_group, user_id
"""

df_03 = pandahouse.read_clickhouse(q, connection=connection)

In [5]:
#T-тест на группах 0(контрольная) и 3(тестовая)

t_test = stats.ttest_ind(df_03[df_03.exp_group == 0].ctr,
                df_03[df_03.exp_group == 3].ctr,
                equal_var=False)
print(f' p-value = {t_test.pvalue}')

 p-value = 1.055849414662529e-43


In [6]:
global_CTR_03 = sum(df_03[df_03.exp_group == 0].likes)/sum(df_03[df_03.exp_group == 0].views)

In [7]:
df_03['linearized_likes'] = df_03.apply(
    lambda x: x['likes'] - x['views']*global_CTR_03, axis=1)

In [ ]:
df_03

In [8]:
t_test_03 = stats.ttest_ind(df_03[df_03.exp_group == 0].linearized_likes,
                df_03[df_03.exp_group == 3].linearized_likes,
                equal_var=False)
print(f' p-value = {t_test_03.pvalue}')

 p-value = 5.491424947968532e-52


Оба т-теста показали значимое отличие.          
p-value в линеарезированном тесте меньше

--------------

In [13]:
# Запрос данных (пользователи, их просмотры и лайки из групп 1 и 2
#  во временной интервал с 01.01.23 по 07.01.23 включительно)

q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM simulator_20230120.feed_actions 
WHERE toDate(time) between '2023-01-01' and '2023-01-07'
    and exp_group in (1,2)
GROUP BY exp_group, user_id
"""

df_12 = pandahouse.read_clickhouse(q, connection=connection)

In [14]:
#T-тест на группах 1(контрольная) и 2(тестовая)

t_test = stats.ttest_ind(df_12[df_12.exp_group == 1].ctr,
                df_12[df_12.exp_group == 2].ctr,
                equal_var=False)
print(f' p-value = {t_test.pvalue}')

 p-value = 0.4780623130874935


In [15]:
global_CTR_12 = sum(df_12[df_12.exp_group == 1].likes)/sum(df_12[df_12.exp_group == 1].views)

In [16]:
df_12['linearized_likes'] = df_12.apply(
    lambda x: x['likes'] - x['views']*global_CTR_12, axis=1)

In [17]:
t_test_12 = stats.ttest_ind(df_12[df_12.exp_group == 1].linearized_likes,
                df_12[df_12.exp_group == 2].linearized_likes,
                equal_var=False)
print(f' p-value = {t_test_12.pvalue}')

 p-value = 9.439432187037712e-10


Т-тесты на пользовательзких CTR и линеаризованных CTR показали разные результаты
